In [1]:
import numpy as np
import pandas as pd
import librosa
import pandas as pd
import csv
import os
import numpy as np
import matplotlib.pyplot as plt
import librosa.display
import IPython.display as ipd

import librosa
import librosa.display

import matplotlib.pyplot as plt
%matplotlib inline

In [5]:
header = 'filename chroma_stft spectral_centroid spectral_bandwidth rolloff zero_crossing_rate tempo flux contrast flatness'
for i in range(1, 21):
    header += f' mfcc{i}'
header += ' label'
header = header.split()
file = open('data.csv', 'w', newline='')
with file:
    writer = csv.writer(file)
    writer.writerow(header)
genres = 'blues classical country disco hiphop jazz metal pop reggae rock'.split()
for g in genres:
    for filename in os.listdir('Data/genres_original/'+g):
        songname = 'Data/genres_original/'+ g + '/' + filename
        y, sr = librosa.load(songname, mono=True, duration=30)
        chroma_stft = librosa.feature.chroma_stft(y=y, sr=sr)
        spec_cent = librosa.feature.spectral_centroid(y=y, sr=sr)
        spec_bw = librosa.feature.spectral_bandwidth(y=y, sr=sr)
        rolloff = librosa.feature.spectral_rolloff(y=y, sr=sr)
        zcr = librosa.feature.zero_crossing_rate(y)
        mfcc = librosa.feature.mfcc(y=y, sr=sr)

        tempo = librosa.beat.tempo(y, sr=sr)[0]

        flux = librosa.onset.onset_strength(y=y, sr=sr)
        contrast = librosa.feature.spectral_contrast(y, sr=sr)
        flatness = librosa.feature.spectral_flatness(y)

        to_append = f'{filename} {np.mean(chroma_stft)} {np.mean(spec_cent)} {np.mean(spec_bw)} {np.mean(rolloff)} {np.mean(zcr)} {np.mean(tempo)} {np.mean(flux)} {np.mean(contrast)} {np.mean(flatness)}'    
        for e in mfcc:
            to_append += f' {np.mean(e)}'
        to_append += f' {g}'
        file = open('data.csv', 'a', newline='')
        with file:
            writer = csv.writer(file)
            writer.writerow(to_append.split())

In [6]:
df = pd.read_csv("data.csv")

In [7]:
df["label"].value_counts()

pop          100
reggae       100
disco        100
rock         100
blues        100
hiphop       100
jazz         100
country      100
metal        100
classical    100
Name: label, dtype: int64

In [11]:
from sklearn import linear_model
from sklearn import svm
from sklearn.pipeline import Pipeline
from sklearn.feature_selection import VarianceThreshold, SelectFromModel
from sklearn.model_selection import GridSearchCV


from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.metrics import accuracy_score, confusion_matrix
from matplotlib import pyplot as plt 
import seaborn as sn

In [17]:
# Scaling the dataset
# and Label encoding the label (genres)
scaler = StandardScaler()
X = scaler.fit_transform(np.array(df.iloc[:, 1:-1], dtype = float))

genre_list = df.iloc[:, -1]
encoder = LabelEncoder()
y = encoder.fit_transform(genre_list)

In [18]:
# Split the dataset with 80% for training set and 20% for test set
X_train, X_test_valid, y_train, y_test_valid= train_test_split(X, y, test_size=0.20, random_state=42)

In [19]:
X_test, X_valid, y_test, y_valid = train_test_split(X_test_valid, y_test_valid, test_size= 0.5, random_state=42)

In [86]:
import numpy
numpy.random.seed(100)
import os
import h5py
import librosa
import itertools
from copy import copy
import numpy as np

from collections import OrderedDict
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix
import tensorflow as tf
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Add
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Activation
from tensorflow.keras.layers import PReLU
from tensorflow.keras.layers import Conv2D
from tensorflow.keras.layers import MaxPooling2D
from tensorflow.keras.layers import AveragePooling2D
from tensorflow.keras.layers import GlobalAveragePooling2D
from tensorflow.keras.layers import GlobalMaxPooling2D
from tensorflow.keras.layers import Dropout
from tensorflow.keras.layers import Flatten
from tensorflow.keras.layers import BatchNormalization
from tensorflow.keras.callbacks import ReduceLROnPlateau
from keras.utils import to_categorical

def conv_block(x, n_filters, pool_size=(2, 2)):
    x = Conv2D(n_filters, (3, 3), strides=(1, 1), padding='same')(x)
    x = Activation('relu')(x)
    x = MaxPooling2D(pool_size=pool_size, strides=pool_size, padding='same')(x)
    x = Dropout(0.25)(x)
    return x

# Model Definition
def create_model(input_shape, num_genres):
    inpt = Input(shape=input_shape)
    x = conv_block(inpt, 16)
    x = conv_block(x, 32)
    x = conv_block(x, 64)
    x = conv_block(x, 128)
    x = conv_block(x, 256)
    
    # Global Pooling and MLP
    x = Flatten()(x)
    x = Dropout(0.5)(x)
    x = Dense(512, activation='relu', 
              kernel_regularizer=tf.keras.regularizers.l2(0.02))(x)
    x = Dropout(0.25)(x)
    predictions = Dense(num_genres, 
                        activation='softmax', 
                        kernel_regularizer=tf.keras.regularizers.l2(0.02))(x)
    
    model = Model(inputs=inpt, outputs=predictions)
    return model
model = create_model(X_train[0].shape, 10)

model.summary()

model.compile(loss=tf.keras.losses.categorical_crossentropy,
              optimizer=tf.keras.optimizers.Adam(),
              metrics=['accuracy'])

Model: "model_10"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_17 (InputLayer)        [(None, 28, 3, 3)]        0         
_________________________________________________________________
conv2d_21 (Conv2D)           (None, 28, 3, 16)         448       
_________________________________________________________________
activation_20 (Activation)   (None, 28, 3, 16)         0         
_________________________________________________________________
max_pooling2d_20 (MaxPooling (None, 14, 2, 16)         0         
_________________________________________________________________
dropout_28 (Dropout)         (None, 14, 2, 16)         0         
_________________________________________________________________
conv2d_22 (Conv2D)           (None, 14, 2, 32)         4640      
_________________________________________________________________
activation_21 (Activation)   (None, 14, 2, 32)         0  